In [1]:
#Linear algebra
import numpy as np 
# data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd 

In [2]:
import csv
df = pd.read_csv('C:/Users/julia/OneDrive/Desktop/UNC_Class_Folder/Team1_FinalProject/Resources/mushrooms_combined_df_clean.csv')
df.head(6)



,Unnamed: 0,Poisonous or Edible,Cap-Shape,Cap-Surface,Cap-Color,Bruises,Odor,Gill-attachment,gill-spacing,Gill-size,...,stalk-surface-below-ring,stalk-color-above-ring,stalk color below ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,p,x,s,n,t,p,f,c,n,...,s,w,w,p,w,o,p,k,s,u
1,1,e,x,f,n,f,n,f,w,b,...,f,w,w,p,w,o,e,k,a,g
2,2,e,x,s,w,f,n,f,w,b,...,f,w,w,p,w,o,e,k,s,g
3,3,e,x,f,g,f,n,f,w,b,...,f,w,w,p,w,o,e,k,s,g
4,4,e,f,f,w,f,n,f,w,b,...,f,w,w,p,w,o,e,n,a,g
5,5,e,x,f,w,f,n,f,w,b,...,f,w,w,p,w,o,e,k,s,g


In [3]:
# Convert all unique values to integers 
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
for col in df.columns:
    df[col] = labelencoder.fit_transform(df[col])
 
df.head()

,Unnamed: 0,Poisonous or Edible,Cap-Shape,Cap-Surface,Cap-Color,Bruises,Odor,Gill-attachment,gill-spacing,Gill-size,...,stalk-surface-below-ring,stalk-color-above-ring,stalk color below ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,1,5,2,4,1,6,1,0,1,...,2,5,5,0,0,1,3,1,3,5
1,1,0,5,0,4,0,5,1,1,0,...,0,5,5,0,0,1,0,1,0,1
2,2,0,5,2,6,0,5,1,1,0,...,0,5,5,0,0,1,0,1,3,1
3,3,0,5,0,3,0,5,1,1,0,...,0,5,5,0,0,1,0,1,3,1
4,4,0,2,0,6,0,5,1,1,0,...,0,5,5,0,0,1,0,2,0,1


In [4]:
#Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#Building everything
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#Evaluation
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
df["Poisonous or Edible"].replace(["e", "p"], [1, 0], inplace= True)

In [6]:
# Create separate object for target variable
y = df["Poisonous or Edible"]
# Create separate object for input features
X = df.iloc[:, 0:1].values

In [7]:
# Divide data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 1234)

In [8]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

4515 1129 4515 1129


In [9]:
pipelines = {
    "rf": make_pipeline(StandardScaler(), RandomForestClassifier(random_state=123)),
    "gb": make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=123))
}

In [10]:
#Hyperparameter Tuning to Optimize performance 
rf_hyperparameters = {
    "randomforestclassifier__n_estimators": [100, 200],
    "randomforestclassifier__max_features": ["auto", "sqrt", 0.33]
}
gb_hyperparameters = {
    "gradientboostingclassifier__n_estimators": [100, 200],
    'gradientboostingclassifier__learning_rate': [0.05, 0.1, 0.2],
    'gradientboostingclassifier__max_depth': [1, 3, 5]
}
hyperparameters = {"rf": rf_hyperparameters,
                   "gb": gb_hyperparameters}

In [11]:
# Create empty dictionary called fitted_models
fitted_models = {}

# Loop through model pipelines, tuning each one and saving it to fitted_models
for name, pipeline in pipelines.items():
    # Create cross-validation object from pipeline and hyperparameters
    model = GridSearchCV(pipeline, hyperparameters[name], cv= 10, n_jobs= -1)
    
    # Fit model on X_train, y_train
    model.fit(X_train, y_train)
    
    # Store model in fitted_models[name] 
    fitted_models[name] = model
    
    # Print '{name} has been fitted'
    print(name, 'has been fitted.')

rf has been fitted.
gb has been fitted.


In [12]:
for name, model in fitted_models.items():
    print(name)
    print("-----------")
    pred = model.predict(X_test)
    print('Acc:', accuracy_score(y_test, pred))
    print("cm:\n", confusion_matrix(y_test, pred))

rf
-----------
Acc: 0.8503100088573959
cm:
 [[607  78]
 [ 91 353]]
gb
-----------
Acc: 0.9069973427812223
cm:
 [[672  13]
 [ 92 352]]
